<a href="https://colab.research.google.com/github/toweliewang/Workshop-NLP/blob/master/Workshop_NLP_Textcat_Toxic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
!/opt/bin/nvidia-smi

Wed Jan 22 13:55:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
%cd /content/drive/My Drive/Colab Notebooks/tsundoku-master/empty/kaggle/

/content/drive/My Drive/Colab Notebooks/tsundoku-master/empty/kaggle


In [10]:
%%html
<style> table {float:left} </style>

In [11]:
!pip install torch tqdm lazyme nltk gensim
!python -m nltk.downloader punkt

  Created wheel for lazyme: filename=lazyme-0.0.23-cp36-none-any.whl size=7931 sha256=5e796c3c37713ccdbf47b68bf979a6e0b1e4f7a49fbca5655f74323777e039bf
  Stored in directory: /root/.cache/pip/wheels/6a/0a/0c/0d199c54c031fd72b1914b7348ffcb7676e6e5caac6a84846d
Successfully built lazyme
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import numpy as np
from tqdm import tqdm

import pandas as pd

from gensim.corpora import Dictionary

import torch
from torch import nn, optim, tensor, autograd
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [0]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = word_tokenize = toktok.tokenize

# Classifying Toxic Comments

Lets apply what we learnt in a realistic task and **fight cyber-abuse with NLP**!

From https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/

> *The threat of abuse and harassment online means that many people stop <br>*
> *expressing themselves and give up on seeking different opinions. <br>*
> *Platforms struggle to effectively facilitate conversations, leading many <br>*
> *communities to limit or completely shut down user comments.*


The goal of the task is to build a model to detect different types of of toxicity:

 - toxic
 - severe toxic
 - threats
 - obscenity
 - insults
 - identity-based hate
 
In this part, you'll be munging the data as how I would be doing it at work. 

Your task is to train a feed-forward network on the toxic comments given the skills we have accomplished thus far.

## Digging into the data...

If you're using linux/Mac you can use these bang commands in the notebook:

```
!pip3 install kaggle
!mkdir -p /content/.kaggle/
!echo '{"username":"natgillin","key":"54ae95ab760b52c3307ed4645c6c9b5d"}' > /content/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*
```

Otherwise, download the data from https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/ 

In [0]:
# !pip3 install kaggle
# !mkdir -p /content/.kaggle/
# !echo '{"username":"natgillin","key":"54ae95ab760b52c3307ed4645c6c9b5d"}' > /content/.kaggle/kaggle.json
# !chmod 600 /content/.kaggle/kaggle.json
# !kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
# !unzip /content/.kaggle/competitions/jigsaw-toxic-comment-classification-challenge/*

In [14]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/tsundoku-master/empty/kaggle/train.csv')
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [15]:
df_train[df_train['threat'] == 1]['comment_text']

79        Hi! I am back again!\nLast warning!\nStop undo...
176       I think that your a Fagget get a oife and burn...
600       I'm also a sock puppet of this account...SUPRI...
802       Fuck you, Smith. Please have me notified when ...
1017      WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...
                                ...                        
157718    bitch \nyou are a fucking hore. you suck dick ...
158717    stupid head \n\nYOur dumb and you are stupid d...
158856    Hey \n\nhey faggot, are you dead yet? or are y...
159029                                  Death to Musulmans!
159400    Shalom \n\nSemite, get the fuck out of here. I...
Name: comment_text, Length: 478, dtype: object

In [21]:
df_train.iloc[3712]['comment_text']

'Please stop. If you continue to ignore our policies by introducing inappropriate pages to Wikipedia, you will be blocked.'

In [0]:
df_train['comment_text_tokenzied'] = df_train['comment_text'].apply(word_tokenize)

In [0]:
# from turicreate import SFrame
# sf_train = SFrame.read_csv('train.csv')

In [0]:
# Just in case your Jupyter kernel dies, save the tokenized text =)

# To save your tokenized text you can do this:
import pickle
with open('train_tokenized_text.pkl', 'wb') as fout:
    pickle.dump(df_train['comment_text_tokenzied'], fout)


In [0]:
# To load it back:
import pickle
with open('train_tokenized_text.pkl', 'rb') as fin:
    df_train['comment_text_tokenzied'] = pickle.load(fin)

# How to get a one-hot?

There are many variants of how to get your one-hot embeddings from the individual columns.

This is one way:

In [25]:
label_column_names = "toxic	severe_toxic	obscene	threat	insult	identity_hate".split()
df_train[label_column_names].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [26]:
torch.tensor(df_train[label_column_names].values).float()

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [27]:
# Convert one-hot to indices of the column.

print(np.argmax(df_train[label_column_names].values, axis=1))

[0 0 0 ... 0 0 0]


In [0]:
class ToxicDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.vocab = Dictionary(texts)
        special_tokens = {'<pad>': 0, '<unk>':1}
        self.vocab = Dictionary(texts)
        self.vocab.patch_with_special_tokens(special_tokens)
        
        self.vocab_size = len(self.vocab)
        
        # Vectorize labels
        self.labels = torch.tensor(labels)
        # Keep track of how many data points.
        self._len = len(texts)
        
        # Find the longest text in the data.
        self.max_len = max(len(txt) for txt in texts)
        
        self.num_labels = len(labels[0])
        
    def __getitem__(self, index):
        vectorized_sent = self.vectorize(self.texts[index])
        # To pad the sentence:
        # Pad left = 0; Pad right = max_len - len of sent.
        pad_dim = (0, self.max_len - len(vectorized_sent))
        vectorized_sent_padded = F.pad(vectorized_sent, pad_dim, 'constant')
        return {'x':vectorized_sent_padded, 
                'y':self.labels[index], 
                'x_len':len(vectorized_sent)}
    
    def __len__(self):
        return self._len
    
    def vectorize(self, tokens):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        # Lets just cast list of indices into torch tensors directly =)
        return torch.tensor(self.vocab.doc2idx(tokens))
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]

In [29]:
# !pip install --upgrade gensim
import gensim
gensim.__version__

'3.8.1'

In [0]:
label_column_names = "toxic	severe_toxic	obscene	threat	insult	identity_hate".split()
toxic_data = ToxicDataset(df_train['comment_text_tokenzied'],
                          df_train[label_column_names].values)

In [31]:
print(toxic_data.texts[123])

['Should', 'say', 'something', 'about', 'his', 'views', 'as', 'an', 'educationalist', 'and', 'socialist', 'political', 'commentator', '.', 'Link', 'to', 'http', ':', '//www.langandlit.ualberta.ca/Fall2004/SteigelBainbridge.html', 'mentions', 'this', 'a', 'bit', '-', 'he', 'stood', 'as', 'an', 'election', 'candidate', 'for', 'Respect', '.']


In [32]:
len(toxic_data)

159571

In [0]:
batch_size = 50
dataloader = DataLoader(dataset=toxic_data, 
                        batch_size=batch_size, shuffle=True)

In [34]:
next(iter(dataloader))

{'x': tensor([[   103,    827,    295,  ...,      0,      0,      0],
         [   245,    123,   1121,  ...,      0,      0,      0],
         [   103,    596,     38,  ...,      0,      0,      0],
         ...,
         [   103,     11,    230,  ...,      0,      0,      0],
         [  4550,     78,    173,  ...,      0,      0,      0],
         [ 68260, 304750,    613,  ...,      0,      0,      0]]),
 'x_len': tensor([ 30, 237, 129,  64, 777, 203, 132, 117,  51,   8, 158,  17,   7,  24,
          15, 124,   5, 178,  43,  22,  13, 129,  30, 208, 415,  26,  43, 109,
          17,  70,  34, 194,  32,  20, 203,  58, 368, 101,  36,  92,  59,  25,
          23,  20,  57,  21,   7,  13,  89,  12, 154,  51, 327,  71,  32,  13,
         207,  71, 101, 108,  25,  39,  34,  59,  41,  44,  36, 322,  14, 139,
          29,  57,  62, 153,  22,  51,  24,  65,  49,  42, 115,  29, 207,  18,
          74,  26, 117,  33, 131,  42,   8,   9, 120,  23,  26, 123,  44,  28,
          37,  66,  26,  75

In [0]:
class FFNet(nn.Module):
    def __init__(self, max_len, num_labels, vocab_size, embedding_size, hidden_dim):
        super(FFNet, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size,
                                       embedding_dim=embedding_size, 
                                       padding_idx=0)
        # The no. of inputs to the linear layer is the 
        # no. of tokens in each input * embedding_size
        self.linear1 = nn.Linear(embedding_size*max_len, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, num_labels)
    
    def forward(self, inputs):
        # We want to flatten the inputs so that we get the matrix of shape.
        # batch_size x no. of tokens in each input * embedding_size
        batch_size, max_len = inputs.shape
        embedded = self.embeddings(inputs).view(batch_size, -1)
        hid = F.relu(self.linear1(embedded))
        out = self.linear2(hid)
        return torch.sigmoid(out)
        

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_size = 100
learning_rate = 0.003
hidden_size = 100


criterion = nn.BCELoss()
# Hint: the CBOW model object you've created.
model = FFNet(toxic_data.max_len, 
              len(label_column_names),
              toxic_data.vocab_size, 
              embedding_size=embedding_size, 
              hidden_dim=hidden_size).to(device)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#model = nn.DataParallel(model)

training_losses = []
num_epochs = 2
for _e in range(num_epochs):
    epoch_loss = []
    for batch in tqdm(dataloader):
        x = batch['x'].to(device)
        y = batch['y'].to(device)
        # Zero gradient.
        optimizer.zero_grad()
        # Feed forward.
        predictions = model(x)
        loss = criterion(predictions, y.float())
        loss.backward()
        optimizer.step()
        epoch_loss.append(float(loss))
        # break
    print(sum(epoch_loss)/len(epoch_loss))
    training_losses.append(sum(epoch_loss)/len(epoch_loss))
    
     

  0%|          | 2/798 [00:00<01:04, 12.27it/s]

0.10653062419671762


100%|██████████| 798/798 [01:05<00:00, 12.31it/s]

0.062275560848172144


In [0]:

def predict(text):
    # Vectorize and Pad.
    vectorized_sent = toxic_data.vectorize(word_tokenize(text)).to(device)
    pad_dim = (0, toxic_data.max_len - len(vectorized_sent))
    vectorized_sent = F.pad(vectorized_sent, pad_dim, 'constant')
    # Forward Propagation.
    # Unsqueeze because model is expecting `batch_size` x `sequence_len` shape.
    outputs = model(vectorized_sent.unsqueeze(0)).squeeze()
    # To get the boolean output, we check if outputs are > 0.5
    return [int(l > 0.5) for l in outputs]
    # What happens if you use torch.max instead? =)
    # return label_column_names[int(torch.max(outputs, dim=1).indices)]

In [82]:
text = df_train.iloc[1017]['comment_text']
print('predict outcome:')
print([n for n in label_column_names if predict(text)[label_column_names.index(n)]==1])

predict outcome:
['toxic', 'obscene', 'insult']


In [71]:
print('test outcome:')
print(df_train.iloc[1017]['comment_text'])
print(df_train.iloc[1017])

test outcome:
WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'LL FIND OUT WHERE YOU LIVE, SODOMIZE YOUR WIFE AND THEN BURN YOUR HOUSE DOWN. FUCK YOU YOU FUCKING QUEER.
id                                                         02c6e41e4b317ac3
comment_text              WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...
toxic                                                                     1
severe_toxic                                                              1
obscene                                                                   1
threat                                                                    1
insult                                                                    1
identity_hate                                                             1
comment_text_tokenzied    [WOULD, N'T, BE, THE, FIRST, TIME, BITCH, ., F...
Name: 1017, dtype: object
